# Yelp and Simple CNN

In [156]:
import numpy as np
import pandas as pd
import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from keras.optimizers import *
from keras.utils import np_utils

In [157]:
from keras import backend as K
K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=32, inter_op_parallelism_threads=32)))

In [158]:
train = pd.read_csv('Neural_Net_Not_Normalized_df.csv')
test = train

In [159]:
train = train.drop(train.columns[train.columns.str.contains('unnamed', case = False)],axis = 1)
train.head(3)

,stars,name_of_month,day_of_week,user_average_stars,user_num_reviews,user_num_friends,business_num_reviews,business_average_stars
0,1,8,2,4.67,6,1,1953,4.0
1,1,8,2,4.67,6,1,84,4.0
2,1,8,2,4.67,6,1,50,4.5


In [160]:
test = test.drop(test.columns[test.columns.str.contains('unnamed', case = False)],axis = 1)
test.head(3)

,stars,name_of_month,day_of_week,user_average_stars,user_num_reviews,user_num_friends,business_num_reviews,business_average_stars
0,1,8,2,4.67,6,1,1953,4.0
1,1,8,2,4.67,6,1,84,4.0
2,1,8,2,4.67,6,1,50,4.5


In [161]:
def encode(train, test):
    label_encoder = LabelEncoder().fit(train.stars)
    labels = label_encoder.transform(train.stars)
    classes = list(label_encoder.classes_)

    train = train.drop('stars', axis=1)

    return train, labels, test, classes

train, labels, test, classes = encode(train, test)

In [162]:
scaler = StandardScaler().fit(train.values)
scaled_train = scaler.transform(train.values)

In [163]:
sss = StratifiedShuffleSplit(test_size=0.1, random_state=23)
for train_index, valid_index in sss.split(scaled_train, labels):
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = labels[train_index], labels[valid_index]

In [164]:
nb_features = 7
nb_class = len(classes)

In [165]:
print(len(X_train))

3764878


In [166]:
X_train_r = np.zeros((len(X_train), nb_features, 3))
print(X_train_r.shape)
X_train_r[:, :, 0] = X_train[:, :nb_features]
print(X_train_r.shape)
#X_train_r[:, :, 1] = X_train[:, nb_features:22] # ERROR
#print(X_train_r.shape)
#X_train_r[:, :, 2] = X_train[:, 22:]
#print(X_train_r.shape)

(3764878, 7, 3)
(3764878, 7, 3)


In [167]:
X_valid_r = np.zeros((len(X_valid), nb_features, 3))
X_valid_r[:, :, 0] = X_valid[:, :nb_features]
#X_valid_r[:, :, 1] = X_valid[:, nb_features:128]
#X_valid_r[:, :, 2] = X_valid[:, 128:]

In [168]:
# model = Sequential()
# model.add(Convolution1D(nb_filter=512, filter_length=1, input_shape=(nb_features, 3)))
# model.add(Activation('relu'))
# model.add(Dropout(0.4))
# model.add(Flatten())
# model.add(Dropout(0.4))
# model.add(Dense(2048, activation='relu'))
# model.add(Dense(1024, activation='relu'))
# model.add(Dense(nb_class))
# model.add(Activation('softmax'))

In [169]:
from keras.layers.advanced_activations import LeakyReLU, PReLU

model = Sequential()
model.add(Convolution1D(nb_filter=512, filter_length=2, input_shape=(nb_features, 3)))
model.add(LeakyReLU(alpha=.003))  
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048))
model.add(LeakyReLU(alpha=.003))  
model.add(Dense(1024))
model.add(LeakyReLU(alpha=.003))  
model.add(Dense(nb_class))
model.add(Activation('softmax'))

/home/kcozart/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(7, 3), filters=512, kernel_size=2)`
  after removing the cwd from sys.path.


In [170]:
y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)

sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer='adadelta', metrics=['accuracy'])

nb_epoch = 1
model.fit(X_train_r, y_train, nb_epoch=nb_epoch, validation_data=(X_valid_r, y_valid), batch_size=300)

/home/kcozart/anaconda3/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3764878 samples, validate on 418320 samples
Epoch 1/1
3764878/3764878 [==============================] - 1118s 297us/step - loss: 0.5023 - acc: 0.7502 - val_loss: 0.5011 - val_acc: 0.7499
